# [Lab1] 데이터 전처리 with SageMaker Processing

이 노트북에서는 SageMaker Processing Job을 사용하여 은행 마케팅 데이터를 전처리합니다.

## 주요 내용
- SageMaker Processing Job 설정
- 데이터 전처리 스크립트 작성
- MLflow를 통한 실험 추적
- 전처리된 데이터를 S3에 저장

## 1. 환경 설정 및 변수 로드

In [ ]:
# 이전 노트북에서 저장한 변수들 로드
%store -r

print("✅ 저장된 변수들을 로드했습니다.")
print(f"   - S3 버킷: {bucket}")
print(f"   - S3 프리픽스: {prefix}")
print(f"   - 리전: {region}")

In [ ]:
# 필수 라이브러리 임포트
import sagemaker
import boto3
import mlflow
import os
from time import gmtime, strftime
from sagemaker.sklearn import SKLearn
from sagemaker.processing import FrameworkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker_studio import Project

# AWS 세션 초기화
boto_session = boto3.Session()
sess = sagemaker.Session()

print("✅ 라이브러리 임포트 및 세션 초기화 완료")

## 2. 전처리 스크립트 준비

SageMaker Processing Job에서 실행될 전처리 스크립트를 작성합니다.

In [ ]:
# 처리 작업을 위한 디렉토리 생성
!mkdir -p processing/requirements

print("✅ 처리 작업 디렉토리 생성 완료")

In [ ]:
%%writefile processing/requirements/requirements.txt
mlflow==2.13.2
sagemaker-mlflow==0.1.0
pandas
numpy
scikit-learn
sagemaker-studio

In [ ]:
%%writefile processing/preprocessing.py
import pandas as pd
import numpy as np
import argparse
import os
import subprocess
import sys

# Requirements 설치
def install_requirements():
    requirements_path = '/opt/ml/processing/input/code/requirements.txt'
    if os.path.exists(requirements_path):
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', '-r', requirements_path])
            print("Requirements installed successfully")
        except subprocess.CalledProcessError as e:
            print(f"Error installing requirements: {e}")
            # 필수 패키지만 개별 설치
            essential_packages = ['mlflow==2.13.2', 'sagemaker-mlflow==0.1.0', 'pandas', 'numpy', 'scikit-learn']
            for package in essential_packages:
                try:
                    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
                except:
                    print(f"Failed to install {package}")
    else:
        print(f"Requirements file not found at {requirements_path}")

# Requirements 설치 실행
install_requirements()

import mlflow
from sklearn.preprocessing import OrdinalEncoder

# MLflow 설정 - 환경 변수에서 가져오기
mlflow_arn = os.getenv('MLFLOW_TRACKING_ARN')
mlflow_run_id = os.getenv('MLFLOW_RUN_ID')
user_profile_name = os.getenv('USER')
domain_id = os.getenv('DOMAIN_ID')

print(f"MLflow ARN: {mlflow_arn}")
print(f"Run ID: {mlflow_run_id}")
print(f"User: {user_profile_name}")
print(f"Domain ID: {domain_id}")

def _parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--filepath', type=str, default='/opt/ml/processing/input/')
    parser.add_argument('--filename', type=str, default='bank-additional-full.csv')
    parser.add_argument('--outputpath', type=str, default='/opt/ml/processing/output/')
    return parser.parse_known_args()

def process_data(args):
    """데이터 전처리 함수"""
    # 데이터 로드
    df = pd.read_csv(os.path.join(args.filepath, args.filename))
    print(f"원본 데이터 크기: {df.shape}")
    
    # 데이터 전처리
    # 1. 점(.)을 언더스코어(_)로 변경
    df = df.replace(regex=r'\.', value='_')
    df = df.replace(regex=r'\_$', value='')
    
    # 2. 새로운 특성 추가
    df["no_previous_contact"] = (df["pdays"] == 999).astype(int)
    df["not_working"] = df["job"].isin(["student", "retired", "unemployed"]).astype(int)
    
    # 3. 불필요한 컬럼 제거
    df = df.drop(['duration', 'emp.var.rate', 'cons.price.idx', 'cons.conf.idx', 'euribor3m', 'nr.employed'], axis=1)
    
    # 4. 범주형 변수 원-핫 인코딩
    df = pd.get_dummies(df)
    print(f"전처리 후 데이터 크기: {df.shape}")
    
    # 5. 훈련/검증/테스트 데이터 분할
    train_data, validation_data, test_data = np.split(
        df.sample(frac=1, random_state=42), 
        [int(0.7 * len(df)), int(0.9 * len(df))]
    )
    
    print(f"훈련 데이터: {train_data.shape}")
    print(f"검증 데이터: {validation_data.shape}")
    print(f"테스트 데이터: {test_data.shape}")
    
    return train_data, validation_data, test_data, df

def save_data(train_data, validation_data, test_data, df, output_path):
    """전처리된 데이터 저장"""
    # 출력 디렉토리 생성
    for subdir in ['train', 'validation', 'test', 'baseline']:
        os.makedirs(os.path.join(output_path, subdir), exist_ok=True)
    
    # 훈련 데이터 저장 (타겟 변수를 첫 번째 컬럼으로)
    pd.concat([train_data['y_yes'], train_data.drop(['y_yes','y_no'], axis=1)], axis=1).to_csv(
        os.path.join(output_path, 'train/train.csv'), index=False, header=False)
    
    # 검증 데이터 저장
    pd.concat([validation_data['y_yes'], validation_data.drop(['y_yes','y_no'], axis=1)], axis=1).to_csv(
        os.path.join(output_path, 'validation/validation.csv'), index=False, header=False)
    
    # 테스트 데이터 저장 (X, y 분리)
    test_data['y_yes'].to_csv(os.path.join(output_path, 'test/test_y.csv'), index=False, header=False)
    test_data.drop(['y_yes','y_no'], axis=1).to_csv(
        os.path.join(output_path, 'test/test_x.csv'), index=False, header=False)
    
    # 베이스라인 데이터 저장
    baseline_path = os.path.join(output_path, 'baseline/baseline.csv')
    df.drop(['y_yes','y_no'], axis=1).to_csv(baseline_path, index=False, header=False)
    
    return baseline_path

if __name__=="__main__":
    args, _ = _parse_args()
    
    # MLflow 설정
    if mlflow_arn:
        mlflow.set_tracking_uri(mlflow_arn)
        mlflow.autolog()
        
        try:
            # 기존 실행 ID 사용 또는 새 실행 생성
            if mlflow_run_id:
                try:
                    client = mlflow.MlflowClient()
                    run_info = client.get_run(mlflow_run_id).info
                    run_context = mlflow.start_run(run_id=mlflow_run_id)
                    print(f"기존 실행 연결: {mlflow_run_id}")
                except Exception as e:
                    print(f"새 실행 생성: {e}")
                    run_context = mlflow.start_run()
            else:
                run_context = mlflow.start_run()
                
            with run_context:
                # 데이터 전처리
                train_data, validation_data, test_data, df = process_data(args)
                
                # MLflow에 파라미터 로깅
                mlflow.log_params({
                    "train_shape": train_data.shape,
                    "validate_shape": validation_data.shape,
                    "test_shape": test_data.shape,
                    "total_features": df.shape[1] - 2  # y_yes, y_no 제외
                })
                
                # 태그 설정
                mlflow.set_tags({
                    'mlflow.user': user_profile_name,
                    'mlflow.source.type': 'PROCESSING_JOB',
                    'sagemaker.domain_id': domain_id,
                    'processing.stage': 'data_preprocessing'
                })
                
                # 데이터 저장
                baseline_path = save_data(train_data, validation_data, test_data, df, args.outputpath)
                
                # MLflow에 아티팩트 로깅
                mlflow.log_artifact(local_path=baseline_path)
                
        except Exception as e:
            print(f"MLflow 오류: {e}")
            # MLflow 없이 데이터 처리
            train_data, validation_data, test_data, df = process_data(args)
            save_data(train_data, validation_data, test_data, df, args.outputpath)
    else:
        print("MLflow ARN이 없습니다. MLflow 없이 처리합니다.")
        train_data, validation_data, test_data, df = process_data(args)
        save_data(train_data, validation_data, test_data, df, args.outputpath)
    
    print("✅ 데이터 전처리 완료")

## 3. 입력 및 출력 경로 설정

SageMaker Processing Job에서 사용할 입력 데이터와 출력 경로를 설정합니다.

In [ ]:
# 입력 데이터를 S3에 업로드
input_source = sess.upload_data(
    './bank-additional/bank-additional-full.csv', 
    bucket=bucket, 
    key_prefix=f'{prefix}/input_data'
)

print(f"✅ 입력 데이터 업로드 완료: {input_source}")

In [ ]:
# 출력 경로 설정
train_path = f"s3://{bucket}/{prefix}/train"
validation_path = f"s3://{bucket}/{prefix}/validation"
test_path = f"s3://{bucket}/{prefix}/test"
baseline_path = f"s3://{bucket}/{prefix}/baseline"

print("✅ 출력 경로 설정 완료:")
print(f"   - 훈련 데이터: {train_path}")
print(f"   - 검증 데이터: {validation_path}")
print(f"   - 테스트 데이터: {test_path}")
print(f"   - 베이스라인: {baseline_path}")

## 4. MLflow 실험 시작

데이터 전처리 과정을 추적하기 위한 MLflow 실험을 시작합니다.

In [ ]:
mlflow.end_run()

In [ ]:
from time import gmtime, strftime
import mlflow

# 프로젝트 설정 (MLflow 제외)
project = Project()
role = project.iam_role
domain_id = project.domain_id
project_id = project.id

# MLflow 직접 설정

print(f"   - MLflow 서버: {mlflow_name}")
mlflow.set_tracking_uri(mlflow_arn)

# 실행 이름 생성
run_suffix = strftime('%d-%H-%M-%S', gmtime())
run_name = f"data-preprocessing-{run_suffix}"

# MLflow 실행 시작
run_id = mlflow.start_run(
    run_name=run_name, 
    description="SageMaker Processing을 사용한 데이터 전처리"
).info.run_id

print(f"✅ MLflow 실행 시작: {run_name}")
print(f"   - 실행 ID: {run_id}")

## 5. SageMaker Processing Job 실행

설정된 전처리 스크립트를 SageMaker Processing Job으로 실행합니다.

In [ ]:

# SageMaker Processing Job 설정
sklearn_processor = FrameworkProcessor(
    estimator_cls=SKLearn,
    framework_version="1.2-1",
    role=role,
    instance_type="ml.m5.large",
    instance_count=1, 
    base_job_name='bank-data-preprocessing',
    env={
        'MLFLOW_TRACKING_ARN': mlflow_arn,
        'MLFLOW_RUN_ID': run_id,
        'USER': user_profile_name,
        'DOMAIN_ID': domain_id
    }
)

print("✅ Processing Job 프로세서 설정 완료")

In [ ]:
# 입력 및 출력 설정
processing_inputs = [
    ProcessingInput(
        source=input_source, 
        destination="/opt/ml/processing/input",
        s3_input_mode="File",
        s3_data_distribution_type="ShardedByS3Key"
    )
]

processing_outputs = [
    ProcessingOutput(
        output_name="train_data", 
        source="/opt/ml/processing/output/train",
        destination=train_path,
    ),
    ProcessingOutput(
        output_name="validation_data", 
        source="/opt/ml/processing/output/validation", 
        destination=validation_path
    ),
    ProcessingOutput(
        output_name="test_data", 
        source="/opt/ml/processing/output/test", 
        destination=test_path
    ),
    ProcessingOutput(
        output_name="baseline_data", 
        source="/opt/ml/processing/output/baseline", 
        destination=baseline_path
    )
]

print("✅ 입력/출력 설정 완료")

In [ ]:
# Processing Job 실행
print("🚀 SageMaker Processing Job 시작...")

sklearn_processor.run(
    inputs=processing_inputs,
    code='processing/preprocessing.py',
    outputs=processing_outputs,
    dependencies=['processing/requirements/requirements.txt'],
    wait=True
)

print("✅ Processing Job 완료!")

## 6. 결과 확인

전처리된 데이터가 올바르게 생성되었는지 확인합니다.

In [ ]:
import pandas as pd
import io

# 훈련 데이터 확인
train_data = sess.read_s3_file(
    bucket=bucket,
    key_prefix=f"{prefix}/train/train.csv"
)

df_train = pd.read_csv(io.StringIO(train_data), header=None)

print("✅ 전처리 결과 확인:")
print(f"   - 훈련 데이터 크기: {df_train.shape}")
print(f"   - 첫 번째 컬럼 (타겟): {df_train[0].value_counts()}")

print("\n📊 훈련 데이터 미리보기:")
display(df_train.head())

In [ ]:
# MLflow 실행 완료 및 메타데이터 추가
mlflow.set_tags({
    'sagemaker.processing_job': sklearn_processor.latest_job.name,
    'sagemaker.domain_id': domain_id,
    'sagemaker.project_id': project_id,
    'processing.status': 'completed'
})

# 현재 실행 ID 저장
current_run_id = mlflow.active_run().info.run_id

mlflow.end_run()

print("✅ MLflow 실행 완료")
print(f"   - Processing Job: {sklearn_processor.latest_job.name}")

In [ ]:
# 다음 노트북에서 사용할 변수들 저장
%store input_source
%store train_path
%store validation_path
%store test_path
%store baseline_path

print("✅ 변수 저장 완료")
print("\n📋 저장된 경로:")
print(f"   - 입력 데이터: {input_source}")
print(f"   - 훈련 데이터: {train_path}")
print(f"   - 검증 데이터: {validation_path}")
print(f"   - 테스트 데이터: {test_path}")
print(f"   - 베이스라인: {baseline_path}")

## ✅ 데이터 전처리 완료

SageMaker Processing Job을 사용한 데이터 전처리가 성공적으로 완료되었습니다!

### 완료된 작업
- ✅ 원본 데이터 전처리 (특성 엔지니어링, 원-핫 인코딩)
- ✅ 훈련/검증/테스트 데이터 분할
- ✅ 전처리된 데이터를 S3에 저장
- ✅ MLflow를 통한 실험 추적

### 다음 단계
이제 `2-training.ipynb` 노트북으로 진행하여 전처리된 데이터로 모델을 훈련할 수 있습니다.

### 생성된 데이터
- **훈련 데이터**: 모델 훈련용 (70%)
- **검증 데이터**: 하이퍼파라미터 튜닝용 (20%)
- **테스트 데이터**: 최종 모델 평가용 (10%)
- **베이스라인**: 모델 모니터링용